In [1]:
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_config('configs/local_next_attribute_regression.yaml', max_workers=2)

tasks = benchmark.get_tasks()
tasks

Total progress: 0it [00:00, ?it/s]

C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\Code\ppm_benchmark_code\ppm_benchmark\Models\BaseDatasetNormalizer.py:23: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  case_starts_df['date'] = case_starts_df["time:timestamp"].dt.to_period('M')
C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\Code\ppm_benchmark_code\ppm_benchmark\Models\BaseDatasetNormalizer.py:40: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  case_stops_df['date'] = case_stops_df["time:timestamp"].dt.to_period('M')
C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\Code\ppm_benchmark_code\ppm_benchmark\Models\BaseDatasetNormalizer.py:23: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  case_starts_df['date'] = case_starts_df["time:timestamp"].dt.to_period('M')
C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\Code\ppm_benchmark_code\ppm_benchmark\Models\BaseDatasetNormal

Identifying attribute drift...:   0%|          | 0/14 [00:00<?, ?it/s]

Finding closest train sequences for test data...:   0%|          | 0/7423 [00:00<?, ?it/s]

Creating evaluation data:   0%|          | 0/174 [00:00<?, ?it/s]

Identifying attribute drift...:   0%|          | 0/15 [00:00<?, ?it/s]

Finding closest train sequences for test data...:   0%|          | 0/6169 [00:00<?, ?it/s]

Creating evaluation data:   0%|          | 0/120 [00:00<?, ?it/s]

['bpi_2015_1_next_timestamp', 'bpi_2015_2_next_timestamp']

In [2]:
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader
from sklearn.ensemble import RandomForestRegressor
from ppm_benchmark.utils.label_encoder import PPMLabelEncoder
from tqdm.notebook import tqdm

loader = BenchmarkLoader()
benchmark = loader.load_from_folder('next_attribute_regression')
tasks = benchmark.get_tasks()
results = dict()

for task_name in tqdm(tasks):
    task = benchmark.load_task(task_name)
    train = task.get_train_data()
    test = task.get_test_data().drop('target', axis=1)
    
    string_cols = train.select_dtypes(include=['object']).columns
    encoder = PPMLabelEncoder()
    train[string_cols] = encoder.fit_transform(train[string_cols])
    string_cols = test.select_dtypes(include=['object']).columns
    test[string_cols] = encoder.transform_with_new_labels(test[string_cols])
    
    X = train.drop('target', axis=1)
    y = train['target']
    
    model = RandomForestRegressor()
    model.fit(X, y)
        
    preds = model.predict(test)
    results[task.name] = preds

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import pickle

with open('next_attribute_regression/test_results.pkl', 'wb') as f:
    pickle.dump(results, f)

## VANAF HIERONDER VOOR TEST.

In [4]:
import pickle
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_folder('next_attribute_regression')
evaluator = benchmark.get_evaluator()

with open('next_attribute_regression/test_results.pkl', 'rb') as f:
    results = pickle.load(f)

In [5]:
for task_name, result in results.items():
    evaluator.add_predictions(task_name, result)
    

In [6]:
evaluator.evaluate()

C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\ppm_benchmark\env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\ppm_benchmark\env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\ppm_benchmark\env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Dell\OneDrive - Universiteit Leiden\Thesis\ppm_benchmark\env\lib\si

[{'task_name': 'bpi_2015_1_next_timestamp',
  'metric': 'MAE',
  'model': 12564.18175891472,
  'baseline': 11010.766797766084},
 {'task_name': 'bpi_2015_1_next_timestamp',
  'metric': 'RMSE',
  'model': 14875.019932277459,
  'baseline': 13468.743113488192},
 {'task_name': 'bpi_2015_1_next_timestamp',
  'metric': 'MSE',
  'model': 221266217.9856517,
  'baseline': 181407041.0571356},
 {'task_name': 'bpi_2015_1_next_timestamp',
  'metric': 'Correlation',
  'model': -0.003536886876933841,
  'baseline': 0},
 {'task_name': 'bpi_2015_2_next_timestamp',
  'metric': 'MAE',
  'model': 13814.676830114584,
  'baseline': 9603.431379025928},
 {'task_name': 'bpi_2015_2_next_timestamp',
  'metric': 'RMSE',
  'model': 16902.47859947415,
  'baseline': 12516.770527176108},
 {'task_name': 'bpi_2015_2_next_timestamp',
  'metric': 'MSE',
  'model': 285693782.80568165,
  'baseline': 156669544.42998448},
 {'task_name': 'bpi_2015_2_next_timestamp',
  'metric': 'Correlation',
  'model': 0.011217158672095827,
  

In [7]:
evaluator.plot_by_fraction_completed('MAE')

AttributeError: 'NextAttributeRegression' object has no attribute 'plot_by_fraction_completed'

In [ ]:
evaluator.plot_by_train_act_distance('MAE')

In [ ]:
evaluator.plot_by_attr_drift_column('MAE')